In [1]:
import pandas as pd

data_path = "data_preprocessed/2_europe_tokenize.csv"
data_out_path = "data_preprocessed/3_selected_features.csv"
df = pd.read_csv(data_path)

In [2]:
df.head()

,id,text3,label
0,de_0982669,leid nicht empfehl leid deckel sehr dunn plast...,1
1,de_0744791,schrott lackschad anschluss kaputt woch allgem...,0
2,de_0342898,krankheit technik toll bild laufend problem to...,1
3,de_0584719,schutzt display nicht sehr gut sitz rundherum ...,3
4,de_0266588,sup stift echt preiswert trotzd sup qualitat f...,3


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=20, max_df=0.95, max_features=5000, ngram_range=(1, 1), tokenizer=lambda x: x.split())
# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(df['text3'])
# Get feature names to view the vocabulary
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

c:\Users\paul-\anaconda3\envs\ml_angew_programm\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['aa' 'aab' 'abbau' ... 'zwischendurch' 'zwischenraum' 'zwischenzeit']


In [4]:
feature_file_path = '../output/tfidf_features.txt'
with open(feature_file_path, 'w') as f:
    for feature in feature_names:
        f.write(f"{feature}\n")

In [5]:
import sys
sys.path.append('../moduls')

from pipeline import feature_selection

In [6]:
from parallel_pandas import ParallelPandas

#initialize parallel-pandas
ParallelPandas.initialize(n_cpu=12, split_factor=1, disable_pr_bar=False)

df['text4'] = df['text3'].p_apply(feature_selection)

FEATURE_SELECTION DONE:   0%|          | 0/167994 [00:00<?, ?it/s]

In [7]:
pd.set_option('display.max_colwidth', None)
df.head()

,id,text3,label,text4
0,de_0982669,leid nicht empfehl leid deckel sehr dunn plastik geriss sehr nicht nochmal kauf,1,leid nicht empfehl leid deckel sehr dunn plastik geriss sehr nicht nochmal kauf
1,de_0744791,schrott lackschad anschluss kaputt woch allgemein all irgendwi nicht wertig spar,0,schrott anschluss kaputt woch allgemein all irgendwi nicht wertig spar
2,de_0342898,krankheit technik toll bild laufend problem ton fernbedien lasst nicht updat dah laufend gross meldung bild fur updat nicht gerat nicht empfehl,1,technik toll bild laufend problem ton fernbedien lasst nicht updat dah laufend gross meldung bild fur updat nicht gerat nicht empfehl
3,de_0584719,schutzt display nicht sehr gut sitz rundherum gut grip nicht bewusst bess les hull metallplatt fur magnethalter eingearbeitet telefon entfern schad find display nicht geschutzt kant vorn nicht uber steh dah stern abzug,3,schutzt display nicht sehr gut sitz rundherum gut grip nicht bewusst bess les hull fur eingearbeitet telefon entfern schad find display nicht geschutzt kant vorn nicht uber steh dah stern abzug
4,de_0266588,sup stift echt preiswert trotzd sup qualitat farb entsprech nicht anblick kapp erwartet einzeln farb mal testet weis ausseh probl bemangeln all farb dunkel sehr farbintensiv paar pastelfarb wunsch zukunft pack komm irgendwann,3,sup stift echt preiswert trotzd sup qualitat farb entsprech nicht kapp erwartet einzeln farb mal weis ausseh probl bemangeln all farb dunkel sehr paar wunsch zukunft pack komm irgendwann


In [8]:
df[df['text4'].apply(lambda x: len(x.split())) < 4].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6257 entries, 25 to 167988
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6257 non-null   object
 1   text3   6257 non-null   object
 2   label   6257 non-null   int64 
 3   text4   6257 non-null   object
dtypes: int64(1), object(3)
memory usage: 244.4+ KB


In [9]:
df_filtered = df[df['text4'].apply(lambda x: len(x.split())) >= 4]

In [10]:
df_filtered[['id', 'text4', 'label']].to_csv(data_out_path, index=False)